In [8]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import json
import os
import time

In [2]:
data_dir = "C:/Users/mhoyl/STAT480/FinalProject/sofia_sensors"

In [6]:
analysis_files = [file for file in os.listdir(data_dir) if file.endswith("_sds011sof.csv")][12:]
analysis_files

['2018-07_sds011sof.csv',
 '2018-08_sds011sof.csv',
 '2018-09_sds011sof.csv',
 '2018-10_sds011sof.csv',
 '2018-11_sds011sof.csv',
 '2018-12_sds011sof.csv',
 '2019-01_sds011sof.csv',
 '2019-02_sds011sof.csv',
 '2019-03_sds011sof.csv',
 '2019-04_sds011sof.csv',
 '2019-05_sds011sof.csv',
 '2019-06_sds011sof.csv']

In [11]:
start_time = time.time()

sofia_dfs = []
for file in analysis_files:
    sofia_dfs.append(pd.read_csv(f"{data_dir}/{file}").drop("Unnamed: 0", axis = 1))
    
print(round((time.time() - start_time)/60, 3), " min")

1.8  min


In [12]:
sofia_dataset = pd.concat(sofia_dfs, axis = 0)

In [13]:
del sofia_dfs

In [21]:
sofia_dataset.drop(["location", "timestamp", "P1", "P2"], axis = 1)

,sensor_id,lat,lon
0,1020,42.647,23.270
1,11560,42.677,23.264
2,11098,42.650,23.364
3,1953,42.704,23.354
4,3511,42.683,23.335
...,...,...,...
5904994,22646,42.726,23.268
5904995,6731,42.674,23.256
5904996,23163,42.692,23.362
5904997,2039,42.686,23.350


In [23]:
sensor_latlong = sofia_dataset.drop(["location", "timestamp", "P1", "P2"], axis = 1).groupby(['sensor_id']).mean().reset_index()
sensor_latlong

,sensor_id,lat,lon
0,739,42.694000,23.336797
1,753,42.626000,23.378000
2,787,42.708000,23.312000
3,797,42.704895,23.312000
4,1018,42.643000,23.409000
...,...,...,...
448,27059,42.669431,23.233351
449,27209,42.668000,23.386000
450,27957,42.658000,23.374000
451,28130,42.634000,23.374000


In [27]:
sensor_latlong.to_csv("sensors_latlong.csv")

In [28]:
with open("Sofia_GeoMapping.json", 'r') as j:
     SofiaGeoMap = json.loads(j.read())

In [93]:
m = folium.Map(location = [42.69, 23.35], zoom_start = 11)

In [94]:
cp = folium.Choropleth(
    geo_data=SofiaGeoMap,
    name="Sofia Districts",
    #data=districtsaq_df,
    #columns=["district", "PM10"],
    key_on="feature.id",
    #fill_color="YlGn",
    fill_color = 'gray',
    line_color = 'black',
    fill_opacity=0.3,
    line_opacity=1,
    highlight = True,
    #tooltip = "District : {0}</i>".format("properties.name"),
    #legend_name="Unemployment Rate (%)",
).add_to(m)

folium.GeoJsonTooltip(['name'],
                      labels = False).add_to(cp.geojson)

folium.LayerControl().add_to(m)

In [95]:
for i in range(sensor_latlong.shape[0]):
    folium.CircleMarker([sensor_latlong["lat"][i], sensor_latlong["lon"][i]], 
                        radius = 6,
                        color = 'black',
                        fill = True,
                        fill_color =  'red', #pressure_flce[i],
                        fill_opacity=1,
                  #popup="<i>sensor {0}</i>".format(sofia_sub["sensor_id"][i]), 
                  tooltip = "sensor : {0}</i>".format(sensor_latlong["sensor_id"][i])).add_to(m)

In [174]:
m

In [157]:
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature

point = Feature(geometry=Point((-46.6318, -23.5523)))
polygon = Polygon(
    [
        [
            (-46.653, -23.543),
            (-46.634, -23.5346),
            (-46.613, -23.543),
            (-46.614, -23.559),
            (-46.631, -23.567),
            (-46.653, -23.560),
            (-46.653, -23.543),
        ]
    ]
)
boolean_point_in_polygon(point, polygon)

True

In [163]:
SofiaGeoMap['features'][0]['id']

'Sredets'

In [210]:
[[tuple(l) for l in SofiaGeoMap['features'][0]['geometry']['coordinates'][0]]]

[[(23.32001903810039, 42.69759410354828),
  (23.31733871376095, 42.68947392504407),
  (23.31972873431276, 42.68947776662517),
  (23.31862982910849, 42.68421377481708),
  (23.32790077203187, 42.68606276733984),
  (23.32964010193777, 42.68405863247198),
  (23.33183496262691, 42.68286454184626),
  (23.33339178165078, 42.68210650253717),
  (23.33532005254862, 42.68227777827062),
  (23.33589576937067, 42.68061102415122),
  (23.33997627517122, 42.67794079069429),
  (23.34459519225232, 42.68112045960795),
  (23.35097351792387, 42.68689614078765),
  (23.3507525258319, 42.68752275007628),
  (23.34886163598074, 42.68734043413015),
  (23.34239422598412, 42.69090592852422),
  (23.34245552531807, 42.69308842917292),
  (23.32001903810039, 42.69759410354828)]]

In [211]:
test_poly = Polygon([[tuple(l) for l in SofiaGeoMap['features'][0]['geometry']['coordinates'][0]]])

In [214]:
testpoint = (float(sensor_latlong[sensor_latlong['sensor_id'] == 11416]['lon']), 
float(sensor_latlong[sensor_latlong['sensor_id'] == 11416]['lat']))

In [208]:
testpoint

(42.69084005582525, 23.329159944122175)

In [178]:
sensor_latlong[sensor_latlong['sensor_id'] == 11416]

,sensor_id,lat,lon
299,11416,42.69084,23.32916


In [213]:
test_poly['coordinates']

[[[23.320019, 42.697594],
  [23.317339, 42.689474],
  [23.319729, 42.689478],
  [23.31863, 42.684214],
  [23.327901, 42.686063],
  [23.32964, 42.684059],
  [23.331835, 42.682865],
  [23.333392, 42.682107],
  [23.33532, 42.682278],
  [23.335896, 42.680611],
  [23.339976, 42.677941],
  [23.344595, 42.68112],
  [23.350974, 42.686896],
  [23.350753, 42.687523],
  [23.348862, 42.68734],
  [23.342394, 42.690906],
  [23.342456, 42.693088],
  [23.320019, 42.697594]]]

In [215]:
boolean_point_in_polygon(Feature(geometry=Point(testpoint)), test_poly)

True

In [200]:
Feature(geometry=Point(testpoint))

{"geometry": {"coordinates": [42.69084, 23.32916], "type": "Point"}, "properties": {}, "type": "Feature"}

In [148]:
ttt = {"district1" : [5468, 890, 6876987, 789],
       "district2" : [28501, 8709, 282, 22]}

In [149]:
ttt.keys()

dict_keys(['district1', 'district2'])

In [150]:
new_dict = {}
for key in ttt.keys():
    for sens in ttt[key]:
        new_dict[str(sens)] = key

In [151]:
new_dict

{'5468': 'district1',
 '890': 'district1',
 '6876987': 'district1',
 '789': 'district1',
 '28501': 'district2',
 '8709': 'district2',
 '282': 'district2',
 '22': 'district2'}

In [152]:
testing = sensor_latlong.copy()

In [153]:
testing['district'] = testing['sensor_id'].astype(str)

In [154]:
testing['district'] = testing['district'].replace(new_dict)

In [155]:
testing[testing['sensor_id'].astype(str).isin(new_dict.keys())]

,sensor_id,lat,lon,district
452,28501,42.714,23.256,district2


In [143]:
"5468" in new_dict.keys()

True

In [147]:
testing['sensor_id'].astype(str)

0        739
1        753
2        787
3        797
4       1018
       ...  
448    27059
449    27209
450    27957
451    28130
452    28501
Name: sensor_id, Length: 453, dtype: object